In [26]:
import warnings
warnings.filterwarnings("ignore")

# DATA MANIPULATION
import numpy as np # linear algebra
import random as rd # generating random numbers
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime # manipulating date formats
from operator import add # elementwise addition

# VIZUALIZATION
import matplotlib.pyplot as plt # basic plotting
import seaborn # for prettier plots

# UNSUPERVISED LEARNING
from sklearn.cluster import AgglomerativeClustering as AggClust # Hierarchical Clustering
from scipy.cluster.hierarchy import ward,dendrogram # Hierarchical Clustering + Dendograms

# TIME SERIES
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller, acf, pacf,arma_order_select_ic


In [28]:
# Reading daily transfers per store
sales=pd.read_csv('../input/transactions.csv')

# Reading store list
stores=pd.read_csv('../input/stores.csv')
stores.type=stores.type.astype('category')

# Adding information about the stores
sales=pd.merge(sales,stores,how='left')

# Reading the holiday and events schedule
holidays=pd.read_csv('../input/holidays_events.csv')

# Formatting the dates properly
sales['date']=sales.date.apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d'))
holidays['date']=holidays.date.apply(lambda x:datetime.datetime.strptime(x, '%Y-%m-%d'))

# Isolating events that do not correspond to holidays
events=holidays.loc[holidays.type=='Event']
holidays=holidays.loc[holidays.type!='Event']

# Extracting year, week and day
sales['year'],sales['week'],sales['day']=list(zip(*sales.date.apply(lambda x: x.isocalendar())))

# Creating a categorical variable showing weekends
sales['dayoff']=[x in [6,7] for x in sales.day]

# Adjusting this variable to show all holidays
for (d,t,l,n) in zip(holidays.date,holidays.type,holidays.locale,holidays.locale_name):
  if t!='Work Day':
    if l=='National':
      sales.loc[sales.date==d,'dayoff']=True
    elif l=='Regional':
      sales.loc[(sales.date==d)&(sales.state==n),'dayoff']=True
    else:
      sales.loc[(sales.date==d)&(sales.city==n),'dayoff']=True
  else:
    sales.loc[(sales.date==d),'dayoff']=False

In [34]:
ts=sales.loc[sales['store_nbr']==47,['date','transactions']].set_index('date')
ts=ts.transactions.astype('float')
plt.figure(figsize=(12,12))
plt.title('Daily transactions in store #47')
plt.xlabel('time')
plt.ylabel('Number of transactions')
plt.plot(ts);

In [35]:
plt.figure(figsize=(12,12))
plt.plot(ts.rolling(window=30,center=False).mean(),label='Rolling Mean');
plt.plot(ts.rolling(window=30,center=False).std(),label='Rolling sd');
plt.legend();

In [36]:
def test_stationarity(timeseries):
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)

test_stationarity(ts)

Results of Dickey-Fuller Test:
Test Statistic                -7.069388e+00
p-value                        4.982766e-10
#Lags Used                     2.200000e+01
Number of Observations Used    1.654000e+03
Critical Value (1%)           -3.434310e+00
Critical Value (5%)           -2.863289e+00
Critical Value (10%)          -2.567701e+00
dtype: float64


In [37]:
plt.figure(figsize=(12,6))
autocorrelation_plot(ts);
plt.figure(figsize=(12,6))
autocorrelation_plot(ts);
plt.xlim(xmax=100);
plt.figure(figsize=(12,6))
autocorrelation_plot(ts);
plt.xlim(xmax=10);

In [38]:
result = arma_order_select_ic(ts,max_ar=10, max_ma=10, ic=['aic','bic'], trend='c', fit_kw=dict(method='css',maxiter=500))
print('The bic prescribes these (p,q) parameters : {}'.format(result.bic_min_order))
print('The aic prescribes these (p,q) parameters : {}'.format(result.aic_min_order))
plt.figure(figsize=(12,6))
plt.subplot(1,2,1)
plt.title('bic results')
seaborn.heatmap(result.bic);
plt.subplot(1,2,2)
plt.title('aic results')
seaborn.heatmap(result.aic);

The bic prescribes these (p,q) parameters : (10, 4)
The aic prescribes these (p,q) parameters : (10, 7)


In [39]:
pdq=(5,0,5)
model = ARIMA(ts, order = pdq, freq='W')
model_fit = model.fit(disp=False,method='css',maxiter=100)
# plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig,axes = plt.subplots(nrows=1, ncols=2,figsize=(12,6))
residuals.plot(ax=axes[0])
residuals.plot(kind='kde',ax=axes[1]);
residuals.describe().T

,count,mean,std,min,25%,50%,75%,max
0,1672.0,0.115749,359.768524,-2751.667551,-183.544339,-22.44322,162.925738,2588.585383


In [40]:
plt.figure(figsize=(12,6))
plt.subplot
plt.plot(ts);
plt.plot(model_fit.fittedvalues,alpha=.7);

In [41]:
forecast_len=30
size = int(len(ts)-forecast_len)
train, test = ts[0:size], ts[size:len(ts)]
history = [x for x in train]
predictions = list()

print('Starting the ARIMA predictions...')
print('\n')
for t in range(len(test)):
    model = ARIMA(history, order = pdq, freq='W');
    model_fit = model.fit(disp=0);
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(float(yhat))
    obs = test[t]
    history.append(obs)
print('Predictions finished.\n')
    

predictions_series = pd.Series(predictions, index = test.index)

Starting the ARIMA predictions...


Predictions finished.



In [42]:
predictions_series.head()

date
2017-07-17    3610.773826
2017-07-18    3535.357786
2017-07-19    3518.011392
2017-07-20    3335.090030
2017-07-21    3910.240834
dtype: float64